# L5d: K-Nearest Neighbors (KNN) classification of a QSAR Chemical Compound Dataset
In this lesson, we will perform a K-Nearest Neighbors (KNN) analysis on a QSAR (Quantitative Structure-Activity Relationship) chemical compound dataset, where the goal is to predict the label (biodegradability) of chemical compounds based on their structural features.

> __Learning Objectives:__
> 
> By the end of this lab, you will be able to:
> * __Prepare KNN-ready data:__ Split the QSAR dataset into training and test partitions and convert each example into a feature-label tuple. Re-encode the original class labels into the binary targets used by the KNN classifier.
> * __Build and sanity-check a kernel KNN classifier:__ Define an RBF kernel and build a weighted kernelized KNN classification model from the training data. Verify the training kernel matrix is positive semidefinite using an eigenvalue check.
> * __Evaluate and tune model performance:__ Compute accuracy, precision, and recall from the confusion matrix on the test set. Compare these metrics across neighbor-count and kernel-width settings during the hyperparameter sweep.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Environment Setup with Include.jl__
>
> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/).

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Data
The dataset we will be using is a QSAR chemical compound dataset, which contains structural features of chemical compounds along with their biodegradability labels. The dataset is available here:

* Mansouri, K., Ringsted, T., Ballabio, D., Todeschini, R., & Consonni, V. (2013). QSAR biodegradation Dataset. UCI Machine Learning Repository. https://doi.org/10.24432/C5H60M.

We built [the `MyQSARBiodegradationDataset()` helper function](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/data/#VLDataScienceMachineLearningPackage.MyQSARBiodegradationDataset) to load the dataset, which is exported by [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). The data is returned as a `DataFrame` and stored in the `originaldataset` variable. 

There are 1055 rows (chemical compounds) and 42 columns (features and labels) in the dataset. The features include various structural descriptors of the chemical compounds, and the label indicates whether the compound is biodegradable (`1`) or not (`2`); see [the openML documentation](https://www.openml.org/search?type=data&id=1494&sort=runs&status=active) for more information on the dataset.

In [2]:
originaldataset = MyQSARBiodegradationDataset() # load the dataset using the helper function

Row,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,Class
,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Float64,Int64,Int64,Int64
1,3.919,2.6909,0,0,0,0,0,31.4,2,0,0,0.0,3.106,2.55,9.002,0,0.96,1.142,0,0,0,1.201,0,0,0,0,1.932,0.011,0,0.0,4.489,0,0,0,0,2.949,1.591,0,7.253,0,0,2
2,4.17,2.1144,0,0,0,0,0,30.8,1,1,0,0.0,2.461,1.393,8.723,1,0.989,1.144,0,0,0,1.104,1,0,0,0,2.214,-0.204,0,0.0,1.542,0,0,0,0,3.315,1.967,0,7.257,0,0,2
3,3.932,3.2512,0,0,0,0,0,26.7,2,4,0,0.0,3.279,2.585,9.11,0,1.009,1.152,0,0,0,1.092,0,0,0,0,1.942,-0.008,0,0.0,4.891,0,0,0,1,3.076,2.417,0,7.601,0,0,2
4,3.0,2.7098,0,0,0,0,0,20.0,0,2,0,0.0,2.1,0.918,6.594,0,1.108,1.167,0,0,0,1.024,0,0,0,0,1.414,1.073,0,8.361,1.333,0,0,0,1,3.046,5.0,0,6.69,0,0,2
5,4.236,3.3944,0,0,0,0,0,29.4,2,4,0,-0.271,3.449,2.753,9.528,2,1.004,1.147,0,0,0,1.137,0,0,0,0,1.985,-0.002,0,10.348,5.588,0,0,0,0,3.351,2.405,0,8.003,0,0,2
6,4.236,3.4286,0,0,0,0,0,28.6,2,4,0,-0.275,3.313,2.522,9.383,1,1.014,1.149,0,0,0,1.119,0,0,0,0,1.98,-0.008,0,10.276,4.746,0,0,0,0,3.351,2.556,0,7.904,0,0,2
7,5.0,5.0476,1,0,0,0,0,11.1,0,3,0,0.0,2.872,0.722,9.657,0,1.092,1.153,0,0,0,1.125,0,0,0,0,2.0,0.446,0,18.375,0.8,0,0,0,1,4.712,4.583,0,9.303,0,0,2
8,4.525,3.8301,0,0,0,0,0,31.6,3,2,0,-0.039,3.418,2.468,9.786,5,0.98,1.142,0,0,0,1.179,0,0,0,0,2.119,-0.002,0,11.115,3.889,0,0,0,0,3.379,2.143,0,7.95,0,0,2
9,4.596,3.0777,0,0,0,0,2,44.4,2,0,0,0.0,2.97,0.875,9.54,0,0.968,1.115,0,0,0,1.328,1,0,0,0,2.175,0.041,0,0.0,1.069,0,0,0,0,3.626,1.917,0,7.939,0,0,2


## Task 1: Let's Split the Data into Training and Test Sets
In this task, we will split the dataset into training and test sets. The distinction between training and test for a KNN classifier is a little different than for other types of classifiers, because KNN is a memory-based learning algorithm. Thus, KNN does not learn a parametric model from the training data; instead, it simply stores the training data and makes predictions based on the similarity of new instances (from the test dataset) to the stored training instances.

> __What is going on in this code block?__
>
> We shuffle row indices, split the dataset into training and test subsets, convert each row into an `(x, y)` named tuple, and map labels from `1/2` to `1/-1`. This representation matches the classifier interface and keeps features and labels together for each example.

Let's save the training and test datasets in the `training::Array{NamedTuple,1}` and `test::Array{NamedTuple,1}` variables, respectively. 

In [3]:
training, test = let

    # initialize -
    total_number_of_examples = nrow(originaldataset); 
    number_of_training_examples = 844; # approximately 80% of the data
    number_of_test_examples = total_number_of_examples - number_of_training_examples; # approximately 20% of the data

    # generate a random permutation of the row indices of the dataset
    shuffled_indices = randperm(total_number_of_examples);
    
    # split the shuffled indices into training and test indices
    training_indices = shuffled_indices[1:number_of_training_examples];
    test_indices = shuffled_indices[number_of_training_examples+1:end];

    # get the training and test datasets using the training and test indices
    training_dataset = originaldataset[training_indices, :];
    test_dataset = originaldataset[test_indices, :];
    
    # make the training into arrays of named tuples
    training = Array{NamedTuple,1}(undef, number_of_training_examples);
    for i ∈ 1:number_of_training_examples
        features = training_dataset[i, 1:end-1] |> values |> collect .|> Float64; # all columns except the last one are features
        label = training_dataset[i, end] == 1 ? 1 : -1; # the last column is the label
        training[i] = (x = features, y = label); # create a named tuple with the features and label
    end

    # make the test into arrays of named tuples
    test = Array{NamedTuple,1}(undef, number_of_test_examples);
    for i ∈ 1:number_of_test_examples
        features = test_dataset[i, 1:end-1] |> values |> collect .|> Float64; # all columns except the last one are features
        label = test_dataset[i, end] == 1 ? 1 : -1; # the last column is the label
        test[i] = (x = features, y = label); # create a named tuple with the features and label
    end
    
    (training, test); # return the training and test datasets as arrays of named tuples
end;

__Are the labels in the training dataset balanced?__ Since we are using a KNN classifier, it is important to have balanced labels in the training dataset. If the labels are imbalanced in our reference dataset, the KNN classifier may be biased towards the majority class, which can lead to poor performance on the minority class. We can check for label balance by looking at the distribution of labels in the training datasets.

In [4]:
let 

    # initialize -
    D = training; # specify what data set we are working with
    number_of_training_examples = length(D); # how many examples are in the training dataset?

    # Fancy! Let's use a list comprehension to count the number of positive and negative labels in the training dataset. The `sum()` function will sum up the number of times the condition is true for each example in the training dataset.
    count_positive_labels = sum(D[i].y == 1 for i ∈ 1:number_of_training_examples); # how many positive labels are in the training dataset?
    count_negative_labels = sum(D[i].y == -1 for i ∈ 1:number_of_training_examples); # how many negative labels are in the training dataset?
    
    # print the results (fraction of positive and negative labels in the training dataset)
    println("Fraction of positive labels in the training dataset: ", count_positive_labels / number_of_training_examples);
    println("Fraction of negative labels in the training dataset: ", count_negative_labels / number_of_training_examples);
end

Fraction of positive labels in the training dataset: 0.6611374407582938
Fraction of negative labels in the training dataset: 0.33886255924170616


__Is this split representative of the overall dataset?__ We can check this by looking at the distribution of labels in the original dataset, and comparing them to the distribution in the training dataset. If the distributions are similar, then we can say that the split is representative of the overall dataset.

In [5]:
let

    # initialize -
    D = originaldataset; # specify what data set we are working with
    number_of_examples = nrow(D); # how many examples are in the training dataset?

    # Let's use a list comprehension to count the number of positive and negative labels in the original dataset. The `sum()` function will sum up the number of times the condition is true for each example in the original dataset.
    count_positive_labels = sum(D[i, end] == 1 for i ∈ 1:number_of_examples); # how many positive labels are in the original dataset?
    count_negative_labels = sum(D[i, end] == 2 for i ∈ 1:number_of_examples); # how many negative labels are in the original dataset?
    
    # print the results (fraction of positive and negative labels in the original dataset)
    println("Fraction of positive labels in the original dataset: ", count_positive_labels / number_of_examples);
    println("Fraction of negative labels in the original dataset: ", count_negative_labels / number_of_examples);
end

Fraction of positive labels in the original dataset: 0.6625592417061611
Fraction of negative labels in the original dataset: 0.33744075829383885


___

## Task 2: Let's look at our KNN Classifier
In this task, we will build and evaluate the KNN classifier that we will be using to make predictions on the test dataset. We'll build [a `MyWeightedKernelizedKNNClassificationModel` model](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.MyWeightedKernelizedKNNClassificationModel) using the `training` dataset using the training dataset as the reference data.

Let's build a kernel function $k:\mathbb{R}^{m}\times\mathbb{R}^{m}\to\mathbb{R}$ to measure similarity. For now, let's make up our own kernel function, and save this function in the `k(x,y)::Function` variable.

In [6]:
k(x,y,γ) = exp(-γ * norm(x-y,2)^2) # RBF kernel

k (generic function with 1 method)

#### Check: Are we using a valid Kernel function?
Let's check to see if the distance (similarity) metric we built is a valid kernel function.

> __Condition:__
>
> A function $k:\mathbb{R}^{m}\times\mathbb{R}^{m}\to\mathbb{R}$ is a _valid kernel function_ if and only if the kernel matrix $\mathbf{K}\in\mathbb{R}^{n\times{n}}$ is positive (semi)definite for all possible choices of the data vectors $\mathbf{v}_i$, where $K_{ij} = k(\mathbf{v}_i, \mathbf{v}_j)$. If $\mathbf{K}$ is positive (semi)definite, then for any real-valued vector $\mathbf{x} \in \mathbb{R}^n$, the kernel matrix $\mathbf{K}$ must satisfy $\mathbf{x}^{\top}\mathbf{K}\mathbf{x} \geq 0$. 

Let's compute the kernel matrix `K::Array{Float64,2}` for a data matrix `X::Array{Float64,2}` using the distance/kernel function `k(x,y)::Function` we built above.

In [7]:
K = let

    D = training; # specify what data set we are working with
    number_of_training_examples = size(D,1);
    number_of_features = D[1].x |> length; # number of features in the dataset
    γ = 0.5; # kernel parameter

    # fill up the feature matrix -
    X = zeros(number_of_training_examples, number_of_features); # initialize a matrix to hold the features
    for i ∈ 1:number_of_training_examples
        X[i,:] = D[i].x; # fill the matrix with the features from the training dataset
    end

    # fill up the kernel matrix -
    K = zeros(number_of_training_examples,number_of_training_examples);
    for i ∈ 1:number_of_training_examples
        vᵢ = X[i,:];
        for j ∈ 1:number_of_training_examples
            vⱼ = X[j,:];
            K[i,j] = k(vᵢ,vⱼ,γ) # compute kernel value
        end
    end
    K
end;

Next, let's check to see if the kernel matrix `K::Array{Float64,2}` is positive (semi)definite by checking if all of its eigenvalues are non-negative.

> __Check:__
>
> For this kernel to be valid, the kernel matrix $\mathbf{K}$ needs to be positive (semi)definite, i.e., all eigenvalues $\lambda_i \geq 0$. We compute the eigenvalues using [`eigvals`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.eigvals) and verify they are all non-negative using [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `all` function](https://docs.julialang.org/en/v1/base/collections/#Base.all-Tuple%7BAny%7D).

Do we blow up? If not, the matrix is PSD for this dataset, which supports using this kernel in this lab.

In [8]:
let
    λ = eigvals(K);
    @assert all(λ .≥ -1e-10) "Kernel matrix is not PSD: min eigenvalue = $(minimum(λ))"
end

Ok, so now let's build the KNN classifier model. There are a few design choices to make, such as the number of neighbors to consider (`K`), the kernel function, and any kernel parameters. 

> __How do we choose the `K` parameter?__
>
> The choice of `K` can significantly affect KNN performance. A common approach is to use cross-validation to select the optimal value (we will do this in the problem set). For this lab, we choose $K = mC + 1$, where $m > 0$ is an adjustable parameter and $C$ is the number of classes in the training dataset. We cam vary `m` to see how it affects the bias-variance tradeoff of the KNN classifier.
>
> In this context, __bias__ means a systematic error from an oversimplified neighborhood vote, and __variance__ means sensitivity to which training points happen to be in the dataset.
> * __Small `K` values:__ The prediction is based on only a few nearby points, so the decision boundary can follow local structure closely. This usually reduces bias, but one noisy or mislabeled neighbor can change the vote, which increases variance.
> * __Large `K` values:__ The prediction averages over many neighbors, so single noisy points have less influence on the vote. This usually reduces variance, but the larger neighborhood can blur local class structure and increase bias.

Can we see this tradeoff in action by varying `K`? Let's test that on the dataset. Next let's consider our second design choice: the kernel width parameter $\gamma$ in the RBF kernel.

> __Gating parameter $\gamma$:__
>
> The parameter $\gamma$ controls the width of the RBF kernel and therefore the smoothness of the decision boundary. Smaller $\gamma$ values produce smoother boundaries, while larger $\gamma$ values produce more complex boundaries.

Let's see how varying $\gamma$ and the number of neighbors `K` affects the performance of our KNN classifier on the test dataset.

In [9]:
model = let
    
    # initialize -
    D = training; # specify what data set we are working with
    number_of_training_examples = size(D,1); # how many examples are in the training dataset?
    number_of_features = D[1].x |> length; # number of features in the dataset
    γ = 0.50; # kernel parameter
    m = 10; # neighborhood size multiple
    C = 2; # number of classes

    # fill up the feature matrix -
    X = zeros(number_of_training_examples, number_of_features); # initialize a matrix to hold the features
    for i ∈ 1:number_of_training_examples
        X[i,:] = D[i].x; # fill the matrix with the features from the training dataset
    end

    # fill up the label vector -
    y = zeros(number_of_training_examples); # initialize a vector to hold the labels
    for i ∈ 1:number_of_training_examples
        y[i] = D[i].y; # fill the vector with the labels from the training dataset
    end

    # build a model -
    model = build(MyWeightedKernelizedKNNClassificationModel, (
        K = (m*C+1), # we look at this many points
        features = X,
        labels = y,
        k = (x,y) -> k(x,y,γ), # RBF kernel similarity metric
    ));

    model; # return the model
end;

### Inference
Now that we have defined a kernel function, and built the model, let's use it to classify our data. We use the KNN classifier from [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). 

> __What is going on in this code block?__
>
> In the code block below, we:
> * Construct [a `MyWeightedKernelizedKNNClassificationModel` model](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.MyWeightedKernelizedKNNClassificationModel) using [a `build(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/factory/). The `model` instance holds the data for the problem, i.e., how many neighbors to look at `K`, and the kernel function $k$.
> * Next, we pass this `model` instance to [the `classify(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.classify) which takes a test feature $\mathbf{z}$ and the classifier `model` instance and returns the predicted label value $\hat{y}$ for the test feature vector $\mathbf{z}$.

We return the predicted labels in `ŷ_KNN` and the actual labels in `y_KNN`.

In [10]:
ŷ_KNN,y_KNN = let

    # Data -
    D = test; # what dataset are we working with?
    number_of_test_examples = size(D,1);
    number_of_features = D[1].x |> length; # number of features in the dataset

     # fill up the feature matrix -
    X = zeros(number_of_test_examples, number_of_features); # initialize a matrix to hold the features
    for i ∈ 1:number_of_test_examples
        X[i,:] = D[i].x; # fill the matrix with the features from the test dataset
    end

    # fill up the label vector (actual labels) -
    y = zeros(number_of_test_examples); # initialize a vector to hold the labels
    for i ∈ 1:number_of_test_examples
        y[i] = D[i].y; # fill the vector with the labels from the test dataset
    end

    # process each vector in the test set, compare that to training (reference), and compute the predicted label -
    ŷ = zeros(number_of_test_examples);  # initialize some storage for the predicted label
    for i ∈ 1:number_of_test_examples
        z = X[i,:]; # get feature vector for test
        ŷ[i] = classify(z,model) # classify the test vector using the training data
    end
 
    # return -
    ŷ,y
end;

### Performance
We can evaluate the binary classifier's performance using various metrics. The central idea is to compare the predicted labels $\hat{y}_{i}$ to the actual labels $y_{i}$ in the `test` dataset and measure wins (when the label is the same) and losses (label is different). This is easily represented in [the confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix).

> __Error analysis using the confusion matrix__
>
> Total mistakes (or mistake percentage) is only part of the story. We should understand whether we are biased toward false positives or false negatives. How many times did we predict a compound was biodegradable when it was not (false positive)? How many times did we predict not biodegradable when it was biodegradable (false negative)? For this we use a confusion matrix. 
>
> The confusion matrix for a binary classifier is typically structured as:
>
>|                     | **Predicted Positive** | **Predicted Negative** |
>|---------------------|------------------------|------------------------|
>| **Actual Positive** | True Positive (TP)     | False Negative (FN)    |
>| **Actual Negative** | False Positive (FP)    | True Negative (TN)     |
>
> From the confusion matrix, we derive three key metrics:
>
> * __Accuracy__ is the fraction of correct predictions overall: $\texttt{accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$. It tells us the overall success rate but can be misleading if classes are imbalanced; a classifier that predicts "negative" for everything might achieve 95% accuracy on an imbalanced dataset.
>
> * __Precision__ answers: "When we predict positive, how often are we right?" $\texttt{precision} = \frac{TP}{TP + FP}$. In our biodegradation context, high precision means fewer false alarms, if you say a compound is biodegradable, it probably is.
>
> * __Recall__ (also called sensitivity) answers: "Of all the true positives, how many did we catch?" $\texttt{recall} = \frac{TP}{TP + FN}$. High recall means we're not missing degradable compounds. 

We compute the __confusion matrix__ to get these counts. We use the [confusion(...) method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/binaryclassification/#VLDataScienceMachineLearningPackage.confusion), which takes actual labels and estimated labels, returning the confusion matrix. We save the confusion matrix in the `CM_KNN::Array{Int64,2}` variable.

In [11]:
CM_KNN = confusion(y_KNN,ŷ_KNN)

2×2 Matrix{Int64}:
 133  8
  64  6

Let's compute the __accuracy__, __precision__, and __recall__ for our KNN classifier using the confusion matrix. We save these metrics in the `accuracy_KNN::Float64`, `precision_KNN::Float64`, and `recall_KNN::Float64` variables, respectively.

In [12]:
accuracy_KNN, precision_KNN, recall_KNN = let

    # compute the confusion matrix -
    CM = CM_KNN; # confusion matrix for KNN classifier

    # compute accuracy, precision, and recall -
    TP = CM[1,1]; # true positives
    TN = CM[2,2]; # true negatives
    FP = CM[2,1]; # false positives
    FN = CM[1,2]; # false negatives

    accuracy = (TP + TN) / sum(CM); # overall accuracy
    precision = TP / (TP + FP); # precision
    recall = TP / (TP + FN); # recall

    (accuracy, precision, recall); # return the metrics
end;

In [13]:
accuracy_KNN, precision_KNN, recall_KNN

(0.6587677725118484, 0.6751269035532995, 0.9432624113475178)

___

## Task 3: Let's do a hyperparameter sweep
In this task, we perform a grid search to see how the number of neighbors `K` and kernel gating parameter $\gamma$ affect classification performance on the test dataset.

> __What is going on in this code block?__
>
> We sweep over several $\left(K,\gamma\right)$ combinations. For each pair, we build a `MyWeightedKernelizedKNNClassificationModel`, classify every test point, compute a confusion matrix, then calculate accuracy, precision, and recall. Finally, we sort all results (by the overall accuracy) so the best settings appear first.

Let's execute the sweep and inspect the top-performing settings. We store the results in the `knn_sweep_results::DataFrame` variable, which contains the `K`, `gamma`, `TP`, `TN`, `FP`, `FN`, `accuracy`, `precision`, and `recall` values for each combination. We sort this DataFrame by accuracy in descending order to see the best-performing hyperparameter settings at the top.

In [14]:
knn_sweep_results = let

    # load data -
    Dtr = training;
    Dte = test;
    ntr = length(Dtr);
    nte = length(Dte);
    nfeatures = length(Dtr[1].x);

    # feature/label arrays for training and test -
    Xtr = zeros(ntr, nfeatures);
    ytr = zeros(ntr);
    for i in 1:ntr
        Xtr[i, :] = Dtr[i].x;
        ytr[i] = Dtr[i].y;
    end

    Xte = zeros(nte, nfeatures);
    yte = zeros(nte);
    for i in 1:nte
        Xte[i, :] = Dte[i].x;
        yte[i] = Dte[i].y;
    end

    # hyperparameter grids -
    K_grid = [1, 3, 5, 9, 15, 21, 31, 41, 51, 71, 101, 151, 201, 301, 401, 501, 701, 843];
    γ_grid = [0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 1.00];

    results = DataFrame(
        K = Int[],
        gamma = Float64[],
        TP = Int[],
        TN = Int[],
        FP = Int[],
        FN = Int[],
        accuracy = Float64[],
        precision = Float64[],
        recall = Float64[]
    );

    # sweep all (K, gamma) combinations -
    for γ in γ_grid
        for K_neighbors in K_grid
            model_local = build(MyWeightedKernelizedKNNClassificationModel, (
                K = K_neighbors,
                features = Xtr,
                labels = ytr,
                k = (x, y) -> k(x, y, γ),
            ));

            # classify test set -
            ŷ = zeros(nte);
            for i in 1:nte
                ŷ[i] = classify(Xte[i, :], model_local);
            end

            # confusion-matrix metrics -
            CM = confusion(yte, ŷ);
            TP = CM[1, 1];
            TN = CM[2, 2];
            FP = CM[2, 1];
            FN = CM[1, 2];

            accuracy = (TP + TN) / sum(CM);
            precision = (TP + FP) == 0 ? 0.0 : TP / (TP + FP);
            recall = (TP + FN) == 0 ? 0.0 : TP / (TP + FN);

            push!(results, (
                K = K_neighbors,
                gamma = γ,
                TP = TP,
                TN = TN,
                FP = FP,
                FN = FN,
                accuracy = accuracy,
                precision = precision,
                recall = recall
            ));
        end
    end

    df = sort(results, :accuracy, rev = true); # sort results by accuracy (descending order);
    df[1:10, :] # return the top 10 results
end;

Next, let's build a table of the results from our hyperparameter search using [the `pretty_table(...)` function exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl).

In [15]:
pretty_table(knn_sweep_results; 
    backend = :text,
    fit_table_in_display_horizontally = false,
    table_format = TextTableFormat(borders = text_table_borders__compact)
);

 ------- --------- ------- ------- ------- ------- ---------- ----------- ----------
      K     gamma      TP      TN      FP      FN   accuracy   precision     recall 
  Int64   Float64   Int64   Int64   Int64   Int64    Float64     Float64    Float64 
 ------- --------- ------- ------- ------- ------- ---------- ----------- ----------
    843       0.1     117      55      15      24   0.815166    0.886364   0.829787
    843      0.25     119      53      17      22   0.815166       0.875   0.843972
    843      0.75     120      51      19      21   0.810427    0.863309   0.851064
    843       0.5     120      50      20      21   0.805687    0.857143   0.851064
    843       1.0     120      50      20      21   0.805687    0.857143   0.851064
    843      0.05     115      54      16      26   0.800948    0.877863   0.815603
    843      0.01     129      35      35      12   0.777251    0.786585   0.914894
     31       0.5     137       7      63       4   0.682464       0.685

__Question:__ Which $(K, \gamma)$ pair appears at the top of `knn_sweep_results`? How does the best configuration's accuracy/recall compare to the baseline configuration from Task 2? If your boss asked you to pick a single configuration for the KNN classifier, which one would you pick and why?
___

## Summary
This notebook builds a weighted kernelized KNN classifier for QSAR biodegradation data and evaluates how neighborhood and kernel choices affect classification quality.

> __Key Takeaways:__
>
> * **Data preparation controls compatibility:** Converting rows into feature-label tuples and using binary target encoding creates the model-ready data structure. This preprocessing step ensures the classifier receives consistent inputs across training and testing.
> * **Kernel checks support model setup:** The RBF kernel passes a positive-semidefinite check on the training kernel matrix in this notebook. This supports using the kernelized similarity calculation for the KNN workflow demonstrated here.
> * **Hyperparameters change performance tradeoffs:** Changing neighbor count and kernel width changes the accuracy, precision, and recall tradeoffs. The sweep results provide a concrete basis for choosing settings before cross-validation.

Next step: use cross-validation to select neighbor count and kernel width before final model reporting.
___